In [3]:
#!/usr/bin/env python3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interpolate
import glob


def find_month(row):
    day = row.DOY
    if day <= 31:
        mon = 1
    if (day >31) & (day <= 60):
        mon = 2
    if (day >60) & (day <= 91):
        mon = 3
    if (day >91) & (day <= 121):
        mon = 4
    if (day >121) & (day <= 152):
        mon = 5
    if (day >152) & (day <= 182):
        mon = 6
    if (day >182) & (day <= 213):
        mon = 7
    if (day >213) & (day <= 244):
        mon = 8
    if (day >244) & (day <= 274):
        mon = 9
    if (day >274) & (day <= 305):
        mon = 10
    if (day >305) & (day <= 335):
        mon = 11
    if (day >335) & (day <= 366):
        mon = 12
        
    return(mon)

def to_bin_latlon(y):
    lat_step = 2.5
    binned_lat = np.floor(y / lat_step) * lat_step
    return(binned_lat)

def cpt(row):
    temp_prof = row.Temp
    cpt = np.nanmin(temp_prof)
    cpt_height = np.nanargmin(temp_prof)/10 + 5
    return(cpt, cpt_height)

def lrt(row):
    temp_prof = row.Temp[:140]
    lr_temp = np.NaN
    lr_height = np.NaN
    temp_prof_1 = temp_prof[1:]
    temp_prof_2 = temp_prof[:-1]
    lr_prof_off_height = -1*(temp_prof_1 - temp_prof_2)/0.1
    lr_prof_off_height_1 = lr_prof_off_height[1:]
    lr_prof_off_height_2 = lr_prof_off_height[:-1]
    lr_prof = np.nanmean([lr_prof_off_height_1, lr_prof_off_height_2], axis=0)

    for idx in np.where(lr_prof <= 2)[0]:
        next_2km_avg_lr = np.nanmean(lr_prof[idx:idx+20])
        if next_2km_avg_lr > 2:
            continue
        elif next_2km_avg_lr <= 2:
            lr_temp = temp_prof[idx + 1]
            lr_height = idx/10 + 5.1
            break
    return(lr_temp, lr_height)
    

profile_files = glob.glob('/usb/cmm_gws/ROMSAF_30daybackgroundRemoval/*')
print('Now loading GPS-RO data.')
tropopause_dataframes = []
for x in profile_files:
    print(x)
    data = np.load(x, allow_pickle=True)
    if np.shape(data)[1] == 7:
        gpsro_full_df = pd.DataFrame(data, columns=['DOY', 'Hour', 'Lat', 'Lon', 'Temp', 'Year', 'Tanom'])
        gpsro_full_df = gpsro_full_df.drop('Hour', axis=1)
    else:
        gpsro_full_df = pd.DataFrame(data, columns=['DOY', 'Lat', 'Lon', 'Temp', 'Year', 'Tanom'])
    
    # Put GPS-RO data into a df
    gpsro_full_df = gpsro_full_df.drop('Tanom', axis=1)

    # Bin data based on location and month and find tropopause characteristics
    gpsro_full_df['Latbin'] = gpsro_full_df.Lat.apply(to_bin_latlon)
    gpsro_full_df['Lonbin'] = gpsro_full_df.Lon.apply(to_bin_latlon)
    gpsro_full_df['Month'] = gpsro_full_df.apply(find_month, axis=1)
    print('Cleaned.')
    gpsro_full_df[['CPT', 'CPT_z']] = gpsro_full_df.apply(cpt, axis=1, result_type="expand")
    #gpsro_full_df[['LRT', 'LRT_z']] = gpsro_full_df.apply(lrt, axis=1, result_type="expand")
    print('Found trop.')
    gpsro_full_df = gpsro_full_df.drop('Temp', axis=1)
    gpsro_full_df = gpsro_full_df.drop('Lat', axis=1)
    gpsro_full_df = gpsro_full_df.drop('Lon', axis=1)
    gpsro_full_df = gpsro_full_df.drop('DOY', axis=1)
    tropopause_dataframes.append(gpsro_full_df)



Now loading GPS-RO data.
/usb/cmm_gws/ROMSAF_30daybackgroundRemoval/ROMSAF_CMM_gravity_wave_anomalies_30daymean_2006.npy
Cleaned.
Found trop.
/usb/cmm_gws/ROMSAF_30daybackgroundRemoval/ROMSAF_CMM_gravity_wave_anomalies_30daymean_2007.npy
Cleaned.
Found trop.
/usb/cmm_gws/ROMSAF_30daybackgroundRemoval/ROMSAF_CMM_gravity_wave_anomalies_30daymean_2008.npy
Cleaned.
Found trop.
/usb/cmm_gws/ROMSAF_30daybackgroundRemoval/ROMSAF_CMM_gravity_wave_anomalies_30daymean_2009.npy
Cleaned.
Found trop.
/usb/cmm_gws/ROMSAF_30daybackgroundRemoval/ROMSAF_CMM_gravity_wave_anomalies_30daymean_2010.npy
Cleaned.
Found trop.
/usb/cmm_gws/ROMSAF_30daybackgroundRemoval/ROMSAF_CMM_gravity_wave_anomalies_30daymean_2011.npy
Cleaned.
Found trop.
/usb/cmm_gws/ROMSAF_30daybackgroundRemoval/ROMSAF_CMM_gravity_wave_anomalies_30daymean_2012.npy
Cleaned.
Found trop.
/usb/cmm_gws/ROMSAF_30daybackgroundRemoval/ROMSAF_CMM_gravity_wave_anomalies_30daymean_2013.npy
Cleaned.
Found trop.
/usb/cmm_gws/ROMSAF_30daybackgroundRemo

In [4]:
def gridder(temp_df):
    lat_vals = np.arange(-30, 30, 2.5)
    lon_vals = np.arange(-180, 180, 2.5)

    cpt_calendar = []
    for month in range(1, 13):
        tempmonth = temp_df[temp_df['Month'] == month]
        cpt_map = []
        for lat_idx in lat_vals:
            lat_df = tempmonth[tempmonth['Latbin'] == lat_idx]
            lat_line_cpt = []
            for lon_idx in lon_vals:
                lon_df = lat_df[lat_df['Lonbin'] == lon_idx]
                if len(lon_df) == 0:
                    cpt = np.NaN
                    cpt_z = np.NaN
                else:
                    cpt = np.nanmean(lon_df.CPT)
                    cpt_z = np.nanmean(lon_df.CPT_z)
                lat_line_cpt.append([cpt, cpt_z])
            cpt_map.append(lat_line_cpt)
        cpt_calendar.append(cpt_map)
    return(cpt_calendar)
    
merged = pd.concat(tropopause_dataframes, axis=0)
for year in range(2006, 2022):
    print('Gridding Year: ', year)
    gpsro_full_df_year = merged[merged['Year'] == year]
    cpt_calendar = gridder(gpsro_full_df_year)
    np.save('/usb/cmm_gws/monthlyDryCPT/{year_str}_CPTandCPZTropics'.format(year_str=str(year)), cpt_calendar)
    #np.save('lrtmaps_{year_str}_c1mamb'.format(year_str=str(year)), lrt_calendar)

Gridding Year:  2006
Gridding Year:  2007
Gridding Year:  2008
Gridding Year:  2009
Gridding Year:  2010
Gridding Year:  2011
Gridding Year:  2012
Gridding Year:  2013
Gridding Year:  2014
Gridding Year:  2015
Gridding Year:  2016
Gridding Year:  2017
Gridding Year:  2018
Gridding Year:  2019
Gridding Year:  2020
